In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import datetime
import time
import statistics
import sklearn
%matplotlib inline
from datetime import date
from scipy import stats as stats
from sklearn import preprocessing
from sklearn import model_selection
from datetime import datetime, timedelta
from tensorflow import keras as keras
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.compose import make_column_transformer

In [2]:
df_time = pd.read_csv("/Users/alex/SAPSICD9.csv")

In [3]:
df_time = df_time.drop_duplicates(subset=['HADM_ID','CHARTTIME', 'ITEM_NAME'], keep='first')

In [5]:
date_mask = "%Y-%m-%d %H:%M:%S"
df_time['CHARTTIME'] = pd.to_datetime(df_time['CHARTTIME'], format= date_mask)
df_time['DISCHTIME'] = pd.to_datetime(df_time['DISCHTIME'], format= date_mask)
df_time['ADMITTIME'] = pd.to_datetime(df_time['ADMITTIME'], format= date_mask)
df_time['HOURS_LEFT'] = round(((df_time['DISCHTIME'] - df_time['CHARTTIME']).dt.total_seconds() / 3600))
df_time['LOS'] = round(((df_time['DISCHTIME'] - df_time['ADMITTIME']).dt.total_seconds() / 3600))

In [6]:
df_values = df_time[['HADM_ID', 'ITEMID', 'CHARTTIME', 'VALUE', 'ITEM_NAME',
       'blood and blood-forming organs', 'circulatory', 'congenital',
       'digestive', 'endocrine, metabolic and immunity',
       'external causes of injury and supplemental classification',
       'genitourinary', 'ill-defined', 'infectious and parasitic',
       'injury and poisoning', 'mental', 'musculoskeletal', 'neoplasms',
       'nervous', 'perinatal period', 'pregnancy, childbirth, and puerperium',
       'respiratory', 'skin and subcutaneous tissue',
       'EMERGENCY', 'URGENT', 'AGE BIN', 'Age Bin_2', 'Age Bin_3', 'Age Bin_4',
       'Age Bin_5', 'Age Bin_6', 'Age Bin_7', 'Age Bin_8', 'Age Bin_9',
       'Age Bin_10', 'Age Bin_11', 'Age Bin_12', 'Age Bin_13', 'Age Bin_14',
       'Age Bin_15', 'Age Bin_16', 'HOURS_IN', 'HOURS_LEFT', 'LOS']]

In [7]:
df_value = df_time[['HADM_ID', 'CHARTTIME', 'ITEM_NAME', 'VALUE']]
#df_values = df_time[['HADM_ID', 'CHARTTIME', 'ITEM_NAME', 'VALUE', 'HOURS_IN', 'SHIFTS_LEFT', 'HOURS_LEFT']]
df_all = pd.DataFrame()
df_all['CHARTTIME'] = df_value['CHARTTIME']
df_all['HADM_ID'] = df_value['HADM_ID']
df_all['HeartRate'] = ""
df_all['SystolicBP'] = ""
df_all['Potassium'] = ""
df_all['Temperature'] = ""
df_all['White_Blood_Cell'] = ""
df_all['Sodium'] = ""
df_all['Bicarbonate'] = ""
df_all['Glasgow_Coma'] = ""
df_all['Birillium'] = ""
df_all['BloodUreaNitrogen'] = ""
df_all['Urine'] = ""
df_all['Mech_Vent'] = ""

In [8]:
HeartRate = df_values[df_values['ITEM_NAME'].str.contains("HeartRate")]
SystolicBP = df_value[df_value['ITEM_NAME'].str.contains("systolicBP")]
Potassium = df_value[df_value['ITEM_NAME'].str.contains("Potassium")]
Temperature = df_value[df_value['ITEM_NAME'].str.contains("temperature")]
White_Blood_Cell = df_value[df_value['ITEM_NAME'].str.contains("White_blood_cell")]
Sodium = df_value[df_value['ITEM_NAME'].str.contains("sodium")]
Bicarbonate = df_value[df_value['ITEM_NAME'].str.contains("bicarbonate")]
Glasgow_Coma = df_value[df_value['ITEM_NAME'].str.contains("glasgow")]
Birillium = df_value[df_value['ITEM_NAME'].str.contains("birillium")]
BloodUreaNitrogen = df_value[df_value['ITEM_NAME'].str.contains("BUN")]
Urine = df_value[df_value['ITEM_NAME'].str.contains("Urine")]
Mech_Vent = df_value[df_value['ITEM_NAME'].str.contains("mech_vent")]

In [9]:
df_all = df_all.merge(HeartRate, on=['HADM_ID', 'CHARTTIME'], how = 'outer')
df_all['HeartRate'] = df_all['VALUE']
df_all = df_all.drop(columns = ['VALUE', 'ITEM_NAME'])

In [10]:
def addvital(VitalList, nameofcol):
    global df_all
    df_all = df_all.merge(VitalList, on=['HADM_ID', 'CHARTTIME'], how = 'outer')
    df_all[nameofcol] = df_all['VALUE']
    df_all = df_all.drop(columns = ['VALUE', 'ITEM_NAME'])
    
addvital(SystolicBP, 'SystolicBP')
addvital(Potassium, 'Potassium')
addvital(Temperature, 'Temperature')
addvital(White_Blood_Cell, 'White_Blood_Cell')
addvital(Sodium, 'Sodium')
addvital(Bicarbonate, 'Bicarbonate')
addvital(Glasgow_Coma, 'Glasgow_Coma')
addvital(Birillium, 'Birillium')
addvital(BloodUreaNitrogen, 'BloodUreaNitrogen')
addvital(Urine, 'Urine')
addvital(Mech_Vent, 'Mech_Vent')

In [11]:
df_all = df_all.drop_duplicates(subset=['HADM_ID','CHARTTIME'], keep='first')

In [12]:
def fordfill(vital, vitalmask):
    df_all[vitalmask] = 0
    df_all.loc[df_all[vital].isna() == False, vitalmask] = 1
    df_all[vital]=df_all.groupby('HADM_ID')[vital].apply(lambda x : x.ffill())
    
fordfill('HeartRate', 'HeartRateMask')
fordfill('SystolicBP', 'SystolicBPMask')
fordfill('Potassium', 'PotassiumMask')
fordfill('Temperature', 'TemperatureMask')
fordfill('White_Blood_Cell', 'White_Blood_CellMask')
fordfill('Sodium', 'SodiumMask')
fordfill('Bicarbonate', 'BicarbonateMask')
fordfill('Glasgow_Coma', 'Glasgow_ComaMask')
fordfill('Birillium', 'BirilliumMask')
fordfill('BloodUreaNitrogen', 'BloodUreaNitrogenMask')
fordfill('Urine', 'UrineMask')
fordfill('Mech_Vent', 'Mech_VentMask')

In [13]:
imputdict = {
    'HeartRate' : 86,
    'SystolicBP' : 120,
    'Potassium' : 4.3,
    'Temperature' : 37.8,
    'White_Blood_Cell' : 9,
    'Sodium' : 140,
    'Bicarbonate' : 26,
    'BloodUreaNitrogen' : 4.7,
    'Glasgow_Coma' : 3,
    'Birillium' : 0.2,
    'Urine' : 150,
    'Mech_Vent' : 0,
    'SHIFTS_LEFT' : 0,
    'HOURS_IN' : 0
}
df_all = df_all.fillna(value=imputdict)

In [14]:
df_all = df_all[['HeartRate', 'SystolicBP', 'Potassium',
       'Temperature', 'White_Blood_Cell', 'Sodium', 'Bicarbonate',
       'Glasgow_Coma', 'Birillium', 'BloodUreaNitrogen', 'Urine', 'Mech_Vent',
        'HeartRateMask', 'SystolicBPMask', 'PotassiumMask', 'TemperatureMask',
       'White_Blood_CellMask', 'SodiumMask', 'BicarbonateMask',
       'Glasgow_ComaMask', 'BirilliumMask', 'BloodUreaNitrogenMask',
       'UrineMask', 'Mech_VentMask', 'HADM_ID', 'blood and blood-forming organs', 'circulatory', 'congenital',
       'digestive', 'endocrine, metabolic and immunity',
       'external causes of injury and supplemental classification',
       'genitourinary', 'ill-defined', 'infectious and parasitic',
       'injury and poisoning', 'mental', 'musculoskeletal', 'neoplasms',
       'nervous', 'perinatal period', 'pregnancy, childbirth, and puerperium',
       'respiratory', 'skin and subcutaneous tissue', 'EMERGENCY', 'URGENT',
       'AGE BIN', 'Age Bin_2', 'Age Bin_3', 'Age Bin_4', 'Age Bin_5',
       'Age Bin_6', 'Age Bin_7', 'Age Bin_8', 'Age Bin_9', 'Age Bin_10',
       'Age Bin_11', 'Age Bin_12', 'Age Bin_13', 'Age Bin_14', 'Age Bin_15',
       'Age Bin_16', 'HOURS_IN', 'HOURS_LEFT', 'LOS']]

In [15]:
df_all = df_all.dropna()

In [16]:
df_all = df_all.drop(df_all[df_all.HOURS_IN > 336].index)

In [17]:
df_all.columns

Index(['HeartRate', 'SystolicBP', 'Potassium', 'Temperature',
       'White_Blood_Cell', 'Sodium', 'Bicarbonate', 'Glasgow_Coma',
       'Birillium', 'BloodUreaNitrogen', 'Urine', 'Mech_Vent', 'HeartRateMask',
       'SystolicBPMask', 'PotassiumMask', 'TemperatureMask',
       'White_Blood_CellMask', 'SodiumMask', 'BicarbonateMask',
       'Glasgow_ComaMask', 'BirilliumMask', 'BloodUreaNitrogenMask',
       'UrineMask', 'Mech_VentMask', 'HADM_ID',
       'blood and blood-forming organs', 'circulatory', 'congenital',
       'digestive', 'endocrine, metabolic and immunity',
       'external causes of injury and supplemental classification',
       'genitourinary', 'ill-defined', 'infectious and parasitic',
       'injury and poisoning', 'mental', 'musculoskeletal', 'neoplasms',
       'nervous', 'perinatal period', 'pregnancy, childbirth, and puerperium',
       'respiratory', 'skin and subcutaneous tissue', 'EMERGENCY', 'URGENT',
       'AGE BIN', 'Age Bin_2', 'Age Bin_3', 'Age Bin_4', 

In [18]:
df_all.to_csv(r'/Users/alex/df_all.csv')

In [143]:
HR = df_all[['HADM_ID','HeartRate', 'HOURS_IN', 'HOURS_LEFT', 'LOS', 'HeartRateMask']]
BP = df_all[['HADM_ID','SystolicBP', 'HOURS_IN', 'HOURS_LEFT', 'LOS', 'SystolicBPMask']]
Pot = df_all[['HADM_ID','Potassium', 'HOURS_IN', 'HOURS_LEFT', 'LOS', 'PotassiumMask']]
Temp = df_all[['HADM_ID','Temperature', 'HOURS_IN', 'HOURS_LEFT', 'LOS', 'TemperatureMask']]
WBC = df_all[['HADM_ID','White_Blood_Cell', 'HOURS_IN', 'HOURS_LEFT', 'LOS', 'White_Blood_CellMask']]
Na = df_all[['HADM_ID','Sodium', 'HOURS_IN', 'HOURS_LEFT', 'LOS', 'SodiumMask']]
Bi = df_all[['HADM_ID','Bicarbonate', 'HOURS_IN', 'HOURS_LEFT', 'LOS', 'BicarbonateMask']]
Glas = df_all[['HADM_ID','Glasgow_Coma', 'HOURS_IN', 'HOURS_LEFT', 'LOS', 'Glasgow_ComaMask']]
Biril = df_all[['HADM_ID','Birillium', 'HOURS_IN', 'HOURS_LEFT', 'LOS', 'BirilliumMask']]
BUN = df_all[['HADM_ID','BloodUreaNitrogen', 'HOURS_IN', 'HOURS_LEFT', 'LOS', 'BloodUreaNitrogenMask']]
Urine = df_all[['HADM_ID','Urine', 'HOURS_IN', 'HOURS_LEFT', 'LOS', 'UrineMask']]
Mech_Vent = df_all[['HADM_ID','Mech_Vent', 'HOURS_IN', 'HOURS_LEFT', 'LOS', 'Mech_VentMask']]

In [144]:
def makeset(data, value):
    length = 336
    hadmlist = data['HADM_ID'].unique()
    #hadmlist = hadmlist[0:1000]
    print(len(hadmlist))
    timeline = pd.DataFrame({'HADM_ID': hadmlist},columns = names)
    mask = pd.DataFrame({'HADM_ID': hadmlist},columns = names)
    
    y = []
    c = []
    for i in range(0,length):
        c.append(i)

    y.append(c)
    for i in range(0, len(hadmlist)-1):
        df = data[data['HADM_ID'] == hadmlist[i]]
        lst = []
        counter = 0
        for j in range(1, (len(df)-1)):
            timein = int(df.iloc[j, 4])
            value = int(df.iloc[j, 1])
            pos = int(df.iloc[j+1, 2]) -1
            left = int(df.iloc[j, 3])
            if pos > length:
                pass
            else:
                timeline.iloc[i, pos] = value
                mask.iloc[i, pos] = 1
                lst.append(left)
            if timein < length:
                timeline.iloc[i, timein] = -1

        y.append(lst)
        
    timeline = timeline.iloc[:, 1:]
    mask = mask.iloc[:, 1:]
    mask = mask.fillna(0)
    timeline = timeline.ffill(axis = 1)
    
    timeline = timeline.fillna(value=value)
    
    return timeline, mask, y

In [145]:
HR = makeset(HR, 86)
BP = makeset(BP, 120)
Pot = makeset(Pot, 4.3)
Temp = makeset(Temp, 37.8)
WBC = makeset(WBC, 9)
Na = makeset(Na, 140)
Bi = makeset(Bi, 26)
Glas = makeset(Glas, 3)
Mech_Vent = makeset(Mech_Vent, 0)
Biril = makeset(Biril, 0.2)
Urine = makeset(Urine, 150)
BUN = makeset(BUN, 4.7)

1000
1000
1000
1000
1000
1000
1000
1000


In [146]:
y = HR[2]

In [147]:
lst = [HR, BP, Pot, Temp, WBC, Na, Bi, Glas]
lstdata = ['HR', 'BP', 'Pot', 'Temp', 'WBC', 'Na', 'Bi', 'Glas']

def makefinal(lst, lstdata):
    binary = pd.DataFrame()
    for i in range(0, 336):
        j = str(i)
        for k in range(0, len(lstdata)):
            mname = ('m' + str(lstdata[k]) + j)
            tname = ('t'+ lstdata[k] + j)
            
            binary[tname] = lst[k][0].iloc[:, i]
            binary[mname] = lst[k][1].iloc[:, i]
    return binary

In [148]:
binary = makefinal(lst, lstdata)

In [149]:
binary = np.asarray(binary)

In [150]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(binary, test_size=0.15, shuffle = False)

print(train.shape[0])
print(test.shape[0])

scaler = StandardScaler()

trnx = scaler.fit_transform(train)
tstx = scaler.transform(test)

850
150


In [152]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
y = pad_sequences(y, padding= 'post' )
trny = y[:850]
tsty = y[850:]

In [153]:
y.shape

(1000, 503)

In [154]:
trny.shape

(850, 503)

In [155]:
trnx.shape

(850, 8048)

In [156]:
trnx = trnx.reshape((trnx.shape[0], 336, 24))
tstx = tstx.reshape((tstx.shape[0], 336, 24))

In [157]:
trnx.shape

(850, 503, 16)